# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [0]:
from keras.datasets import cifar10
import pandas as pd
import numpy as np
from keras.utils import np_utils
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

from keras.models import Sequential
from keras.layers.core import Dense,Dropout,Activation,Flatten
from keras.layers.convolutional import Conv2D,MaxPooling2D

from sklearn.datasets import fetch_20newsgroups

In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [5]:
x_train.shape

(50000, 32, 32, 3)

In [6]:
y_train.shape

(50000, 1)

In [7]:
type(y_train)

numpy.ndarray

In [0]:
y_test_flat = (y_test < 5).flatten()
y_test_flat = (y_test >= 5).flatten()
y_train_flat = (y_train < 5).flatten()
y_train_flat = (y_train >= 5).flatten()


In [0]:
x_test_less5 = x_test[y_test_flat]
x_test_gt5 = x_test[y_test_flat]
x_train_less5 = x_train[y_train_flat]
x_train_gt5 = x_train[y_train_flat]



In [0]:
y_test_less5 = y_test[y_test < 5]
y_test_gt5 = y_test[y_test >= 5] - 5
y_train_less5 = y_train[y_train < 5]
y_train_gt5 = y_train[y_train >= 5] - 5


In [0]:
x_train_less5 = (x_train_less5.astype('float32'))/255
x_test_less5 = (x_test_less5.astype('float32'))/255
x_train_gt5 = (x_train_gt5.astype('float32'))/255
x_test_gt5 = (x_test_gt5.astype('float32'))/255

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
y_test_less5 = np_utils.to_categorical(y_test_less5, 5, dtype='float32')
y_train_less5 = np_utils.to_categorical(y_train_less5, 5, dtype='float32')
y_test_gt5 = np_utils.to_categorical(y_test_gt5, 5, dtype='float32')
y_train_gt5 = np_utils.to_categorical(y_train_gt5, 5, dtype='float32')

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [13]:
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=x_train.shape[1:], activation='relu'))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
model.fit(x_train_less5, y_train_less5, batch_size=32, epochs=50, validation_data=(x_test_less5, y_test_less5))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 25000 samples, validate on 5000 samples
Epoch 1/50
11392/25000 [============>.................] - ETA: 1:02 - loss: 1.6118 - acc: 0.2041

In [0]:
model.evaluate(x_test_less5,y_test_less5)

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [0]:
model.summary()

In [0]:
model.layers

In [0]:
for layer in model.layers[:9]:
    layer.trainable = False
    print(layer.trainable, layer)

### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [0]:
model.fit(x_train_gt5, y_train_gt5, validation_data=(x_test_gt5, y_test_gt5), epochs=10, batch_size=32)

In [0]:
model.evaluate(x_test_gt5, y_test_gt5)

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [0]:
train_news = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
test_news = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [0]:
train_news.target

In [0]:
train_news.target_names

In [0]:
train_news.data[0:5]

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
vect = TfidfVectorizer(ngram_range=(1,3), min_df=2, max_df=0.5,stop_words='english')

In [0]:
x_train_dt = vect.fit_transform(train_news.data)
x_test_dt = vect.transform(test_news.data)

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [0]:
from sklearn.linear_model import LogisticRegression
LReg = LogisticRegression()


In [0]:
LReg.fit(x_train_dt, train_news.target)
ytrainpred = LReg.predict(x_train_dt)

In [0]:
ypred = LReg.predict(x_test_dt)

In [0]:
print(accuracy_score(ypred, test_news.target))
print(accuracy_score(ytrainpred, train_news.target))